<h1 align="center"><font color="yellow">LangChain 3: Memória ChatBot para ChatGPT, Davinci + outros LLMs</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Neste script seguimos aprimorando nossos conhecimentos com o Framework LangChain com o [James Briggs](https://www.pinecone.io/learn/langchain-conversational-memory/).

# Contextualizando

A `memória conversacional` é como um `chatbot` pode responder a várias consultas de maneira semelhante a um bate-papo. Ele permite uma conversa coerente e, sem ele, cada consulta seria tratada como uma entrada totalmente independente, sem considerar as interações anteriores. A `memória` permite que um Large Language Model (`LLM`) lembre-se de interações anteriores com o usuário. <font color="orange">Por padrão, os LLMs são sem estado — significando que cada consulta recebida é processada independentemente de outras interações.</font> A única coisa que existe para um agente sem estado é a entrada atual, nada mais. Existem muitos aplicativos em que lembrar as interações anteriores é muito importante, como os `chatbots`. A `memória conversacional` nos permite fazer isso. Existem várias maneiras de implementar a memória conversacional. No contexto do `LangChain`, todos eles são construídos sobre o ``ConversationChain``.

# Memória Conversacional

Começaremos importando todas as bibliotecas que usaremos neste exemplo:

In [ ]:
#%pip install -qU langchain openai tiktoken

In [1]:
import inspect

from getpass import getpass
from langchain import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory, 
                                                  ConversationSummaryMemory, 
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

In [2]:
# Isto é quando usas o arquivo .env:
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 

Carregando a minha chave Key:  True


In [3]:
# Aqui instanciamos o Modelo que vamos usar:

llm = OpenAI(
    temperature=0, 
    openai_api_key=Eddy_API_KEY_OpenAI,
    model_name='text-davinci-003'  # Podemos usar o LLM como: 'gpt-3.5-turbo'
)

<font color="orange">Posteriormente, faremos uso de uma função utilitária `count_tokens`. Isso nos permitirá contar o número de tokens que estamos usando para cada chamada. Nós o definimos assim:</font>

In [4]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Gastou um total de {cb.total_tokens} Tokens')

    return result

<font color="pink">Agora vamos mergulhar na `memória conversacional`:</font>

# O que é memória?

Memória é a capacidade de um agente de lembrar interações anteriores com o usuário (`pense em chatbots`)

A definição oficial de `memória` é a seguinte:

<font color="orange">Por padrão, `Chains` e `Agents` são sem estado, o que significa que eles tratam cada consulta recebida de forma independente. Em algumas aplicações (sendo os `chatbots` um ÓTIMO exemplo), é muito importante lembrar as interações anteriores, tanto a curto prazo, quanto a longo prazo. O conceito de `“Memória”` existe para fazer exatamente isso.</font>

Como veremos, embora isso pareça muito simples, existem várias maneiras diferentes de implementar esse recurso de memória.

Antes de nos aprofundarmos nos diferentes módulos de memória que a biblioteca oferece, vamos apresentar a cadeia que usaremos para esses exemplos: a `ConversationChain`.

Como sempre, ao entender uma chain, é interessante dar uma olhada em seu `prompt` primeiro e depois dar uma olhada em seu método `._call`. Como vimos no capítulo sobre `chains`, podemos verificar o `prompt` acessando o `template` dentro do atributo prompt.

In [5]:
conversation = ConversationChain(
    llm=llm, 
)


In [6]:
print(conversation.prompt.template)


The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


O que se segue é uma conversa amigável entre um `humano` e uma `IA`. A IA é falante e fornece muitos detalhes específicos de seu contexto. Se a IA não souber a resposta para uma pergunta, ela diz com sinceridade que não sabe.
```
Current conversaton:
{history}
Human: {input}
AI:
```
Interessante! Portanto, o prompt desta cadeia está dizendo para conversar com o usuário e tentar dar respostas verdadeiras. Se olharmos de perto, há um novo componente no `prompt` que não vimos quando estávamos mexendo no `LLMMathChain`: history. É aqui que nossa memória entrará em ação.

O que essa cadeia está fazendo com esse prompt? Vamos dar uma olhada.

In [10]:
# Tudo junto:
print(inspect.getsource(conversation._call), inspect.getsource(conversation.apply))

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        return self.apply([inputs])[0]
     def apply(self, input_list: List[Dict[str, Any]]) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        response = self.generate(input_list)
        return self.create_outputs(response)



In [8]:
print(inspect.getsource(conversation._call))


    def _call(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        return self.apply([inputs])[0]



In [9]:
print(inspect.getsource(conversation.apply))

    def apply(self, input_list: List[Dict[str, Any]]) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        response = self.generate(input_list)
        return self.create_outputs(response)



<font color="orange">Nada realmente mágico acontecendo aqui, apenas uma passagem direta por um `LLM`. Na verdade, essa cadeia herda esses métodos diretamente do `LLMChain` sem nenhuma modificação:</font>

In [11]:
# Tudo junto:
print(inspect.getsource(LLMChain._call), inspect.getsource(LLMChain.apply))


    def _call(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        return self.apply([inputs])[0]
     def apply(self, input_list: List[Dict[str, Any]]) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        response = self.generate(input_list)
        return self.create_outputs(response)



In [12]:
print(inspect.getsource(LLMChain._call))

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        return self.apply([inputs])[0]



In [13]:
print(inspect.getsource(LLMChain.apply))

    def apply(self, input_list: List[Dict[str, Any]]) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        response = self.generate(input_list)
        return self.create_outputs(response)



<font color="orange">Então, basicamente, essa cadeia combina uma `entrada do usuário` com o `histórico da conversa` para gerar uma resposta significativa (e, com sorte, verdadeira).

Agora que entendemos o básico da cadeia que usaremos, podemos entrar na memória. Vamos mergulhar!</font>

# Tipos de Memória

<font color="orange">Nesta seção, revisaremos vários tipos de memória e analisaremos os `prós` e `contras` de cada um, para que você possa escolher o melhor para o seu caso de uso.</font>

## <font color="red">Tipo de memória Nº 1:</font> ConversationBufferMemory

O `ConversationBufferMemory` faz exatamente o que seu nome sugere: ele mantém um buffer dos trechos da conversa anterior como parte do contexto no prompt.

`Recurso principal:` a memória do buffer de conversação mantém as partes anteriores da conversa completamente inalteradas, em sua forma bruta.

In [15]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)


<font color="orange">Passamos um `prompt` de `usuário` para `ConversationBufferMemory` da seguinte forma:</font>

In [33]:

conversation_buf("Bom dia AI")

{'input': 'Bom dia AI',
 'history': 'Human: Bom dia AI\nAI:  Bom dia! Como posso ajudar?\nHuman: Bom dia AI\nAI:  Bom dia novamente! O que posso fazer por você?\nHuman: Meu interesse aqui é explorar o potencial de integração de Large Language Models com conhecimento externo.\nAI:  Entendo. Estou familiarizado com Large Language Models e como eles podem ser usados para melhorar a compreensão de linguagem natural. Estou interessado em saber como esses modelos podem ser integrados com conhecimento externo. Você pode me dar alguns exemplos de como isso pode ser feito?\nHuman: Eu só quero analisar as diferentes possibilidades. O que você pode pensar?\nAI:  Existem várias maneiras de integrar Large Language Models com conhecimento externo. Uma maneira é usar modelos de aprendizado profundo para incorporar conhecimento externo aos modelos de linguagem. Outra maneira é usar modelos de memória para armazenar e recuperar informações externas. Além disso, também é possível usar modelos de atenção

Esta chamada usou um total de "n" tokens, mas não podemos ver isso acima. Se quisermos contar o número de tokens sendo usados, basta passar nosso objeto da cadeia de conversação e a mensagem que gostaríamos de inserir por meio da função `count_tokens` que definimos anteriormente:

In [17]:
count_tokens(conversation_buf, "Bom dia AI")

Spent a total of 105 tokens


' Bom dia novamente! O que posso fazer por você?'

In [18]:
count_tokens(
    conversation_buf, 
    "Meu interesse aqui é explorar o potencial de integração de Large Language Models com conhecimento externo."
)

Spent a total of 235 tokens


' Entendo. Estou familiarizado com Large Language Models e como eles podem ser usados para melhorar a compreensão de linguagem natural. Estou interessado em saber como esses modelos podem ser integrados com conhecimento externo. Você pode me dar alguns exemplos de como isso pode ser feito?'

In [19]:
count_tokens(
    conversation_buf,
    "Eu só quero analisar as diferentes possibilidades. O que você pode pensar?"
)

Spent a total of 414 tokens


' Existem várias maneiras de integrar Large Language Models com conhecimento externo. Uma maneira é usar modelos de aprendizado profundo para incorporar conhecimento externo aos modelos de linguagem. Outra maneira é usar modelos de memória para armazenar e recuperar informações externas. Além disso, também é possível usar modelos de atenção para direcionar a atenção para informações externas relevantes.'

In [20]:
count_tokens(
    conversation_buf, 
    "Quais tipos de fonte de dados podem ser usados para fornecer contexto ao modelo?"
)

Spent a total of 594 tokens


' Existem várias fontes de dados que podem ser usadas para fornecer contexto aos modelos de linguagem. Estas incluem bases de dados de conhecimento, como o Wikidata, fontes de dados estruturadas, como o Freebase, e fontes de dados não estruturadas, como notícias, documentos e conversas. Além disso, também é possível usar dados de aprendizado profundo, como imagens e vídeos, para fornecer contexto aos modelos.'

In [21]:
count_tokens(
    conversation_buf, 
    "Qual é o meu objetivo novamente?"
)

Spent a total of 645 tokens


' Seu objetivo é explorar o potencial de integração de Large Language Models com conhecimento externo.'

Nosso `LLM` com `ConversationBufferMemory` pode lembrar claramente as interações anteriores na conversa. Vamos dar uma olhada em como o `LLM` está salvando nossa conversa anterior. Podemos fazer isso acessando o atributo `.buffer` para `.memory` em nossa cadeia.

In [22]:
print(conversation_buf.memory.buffer)

Human: Bom dia AI
AI:  Bom dia! Como posso ajudar?
Human: Bom dia AI
AI:  Bom dia novamente! O que posso fazer por você?
Human: Meu interesse aqui é explorar o potencial de integração de Large Language Models com conhecimento externo.
AI:  Entendo. Estou familiarizado com Large Language Models e como eles podem ser usados para melhorar a compreensão de linguagem natural. Estou interessado em saber como esses modelos podem ser integrados com conhecimento externo. Você pode me dar alguns exemplos de como isso pode ser feito?
Human: Eu só quero analisar as diferentes possibilidades. O que você pode pensar?
AI:  Existem várias maneiras de integrar Large Language Models com conhecimento externo. Uma maneira é usar modelos de aprendizado profundo para incorporar conhecimento externo aos modelos de linguagem. Outra maneira é usar modelos de memória para armazenar e recuperar informações externas. Além disso, também é possível usar modelos de atenção para direcionar a atenção para informações 

<font color="orange">Legal! Portanto, cada parte de nossa conversa foi explicitamente gravada e enviada ao `LLM` no `prompt`.</font>

## <font color="red">Tipo de memória Nº 2:</font> ConversationSummaryMemory

<font color="pink">O problema com `ConversationBufferMemory` é que conforme a conversa progride, a contagem de tokens de nosso histórico de contexto aumenta. Isso é problemático porque podemos maximizar nosso `LLM` com um `prompt` muito grande para ser processado.</font>

Digite `ConversationSummaryMemory`.

Mais uma vez, podemos inferir a partir do nome o que está acontecendo. Manteremos um resumo de nossos trechos de conversas anteriores como nosso histórico. `Como vamos resumi-los?` LLM para o resgate.

`Funcionalidade principal:` a memória de resumo da conversação mantém os trechos anteriores da conversa de forma resumida, onde a sumarização é realizada por um `LLM`.

Nesse caso, precisamos enviar o llm para nosso construtor de memória para ativar sua capacidade de resumo.

In [23]:
conversation_sum = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=llm)
)

<font color="orange">Quando temos um LLM, sempre temos um `prompt` 😎 Vamos ver o que está acontecendo dentro da nossa memória de resumo da conversa:</font>

In [24]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


<font color="orange">Legal! Assim, cada nova interação é resumida e anexada a um resumo em execução como a memória de nossa cadeia. Vamos ver como isso funciona na prática!</font>

In [25]:
# Sem count_tokens, chamaríamos `conversation_sum("Bom dia AI!")`
# Mas vamos manter o controle de nossos Tokens:
count_tokens(
    conversation_sum, 
    "Bom dia AI!"
)

Spent a total of 281 tokens


' Bom dia! Como posso ajudar?'

In [26]:
count_tokens(
    conversation_sum, 
    "Meu interesse aqui é explorar o potencial de integração de Large Language Models com conhecimento externo."
)

Spent a total of 629 tokens


' Entendo. Estou familiarizado com Large Language Models e como eles podem ser usados para melhorar a compreensão de linguagem natural. Estou interessado em saber como esses modelos podem ser integrados com conhecimento externo. Existe alguma abordagem específica que você está considerando?'

In [27]:
count_tokens(
    conversation_sum, 
    "Eu só quero analisar as diferentes possibilidades. O que você pode pensar?"
)

Spent a total of 963 tokens


' Entendo. Existem várias abordagens para integrar modelos de linguagem grandes com conhecimento externo. Uma abordagem comum é usar modelos de linguagem grandes para gerar representações de palavras que podem ser usadas para acessar conhecimento externo. Outra abordagem é usar modelos de linguagem grandes para gerar representações de frases que podem ser usadas para acessar conhecimento externo. Além disso, existem abordagens que combinam essas duas abordagens.'

In [28]:
count_tokens(
    conversation_sum, 
    "Quais tipos de fonte de dados podem ser usados para fornecer contexto ao modelo?"
)

Spent a total of 1113 tokens


' Existem várias fontes de dados que podem ser usadas para fornecer contexto ao modelo de linguagem grande, como bases de dados de conhecimento, corpora de texto, bases de dados de imagens, bases de dados de áudio e vídeo, entre outras. Além disso, também é possível usar técnicas de aprendizado de máquina para extrair informações relevantes de fontes de dados não estruturadas.'

In [29]:
count_tokens(
    conversation_sum, 
    "Qual é o meu objetivo novamente?"
)

Spent a total of 1088 tokens


' Seu objetivo é analisar as diferentes possibilidades de integrar modelos de linguagem de grande porte com conhecimento externo para melhorar a compreensão de linguagem natural.'

In [30]:
print(conversation_sum.memory.buffer)



The human greets the AI with "Bom dia!" and the AI responds with "Bom dia! Como posso ajudar?". The human then expresses interest in exploring the potential of integrating Large Language Models with external knowledge, to which the AI responds that it is familiar with Large Language Models and how they can be used to improve natural language understanding. The AI is interested in knowing how these models can be integrated with external knowledge and asks if there is a specific approach the human is considering. The human responds that they just want to analyze the different possibilities, and the AI explains that there are various approaches to integrating Large Language Models with external knowledge, such as using the models to generate word representations to access external knowledge, using the models to generate phrase representations to access external knowledge, or combining both approaches. The human then asks what types of data sources can be used to provide context to the mo

<font color="orange">Você deve estar se perguntando . . . se a contagem agregada de tokens é maior em cada chamada aqui do que no exemplo do buffer, `por que deveríamos usar esse tipo de memória?` Bem, se verificarmos o buffer, perceberemos que, embora estejamos usando mais tokens em cada instância de nossa conversa, nosso histórico final é mais curto. Isso nos permitirá ter muito mais interações antes de `atingirmos o tamanho máximo do nosso prompt`, tornando nosso `chatbot` mais robusto para conversas mais longas.

Podemos contar o número de tokens sendo usados (`sem fazer uma chamada para OpenAI`) usando o tokenizer `tiktoken` da seguinte forma:</font>

In [31]:
# Inicializamos o tokenizer
tokenizer = tiktoken.encoding_for_model('text-davinci-003')

# Mostra o número de TOKENS para a memória usada por cada tipo de memória
print(
    f'Comprimento da conversação da memória do buffer (ConversationBufferMemory): {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Comprimento da conversa de memória resumida (ConversationSummaryMemory): {len(tokenizer.encode(conversation_sum.memory.buffer))}'
)


Comprimento da conversação da memória do buffer (ConversationBufferMemory): 591
Comprimento da conversa de memória resumida (ConversationSummaryMemory): 256


<font color="yellow">Nota prática:</font> 

os modelos `text-davinci-003` e `gpt-3.5-turbo` têm uma grande [contagem máxima de tokens](https://platform.openai.com/docs/api-reference/completions/create#completions/create-max_tokens) de `4096 tokens` entre `prompt` e `resposta`.

## <font color="red">Tipo de memória Nº 3:</font> ConversationBufferWindowMemory

Outra ótima opção para esses casos é a `ConversationBufferWindowMemory`, onde manteremos algumas das últimas interações em nossa memória, mas descartaremos intencionalmente as mais antigas - `memória de curto prazo`, se desejar. Aqui, a contagem agregada de tokens e a contagem de tokens por chamada cairão visivelmente. **Vamos controlar esta janela com o parâmetro `k`**.

`Recurso principal:` <font color="orange">a memória da janela do buffer de conversação mantém as partes mais recentes da conversa em formato bruto.</font>

In [34]:
conversation_bufw = ConversationChain(
    llm=llm, 
    memory=ConversationBufferWindowMemory(k=1)
)


In [35]:
count_tokens(
    conversation_bufw, 
    "Bom dia AI!"
)


Spent a total of 78 tokens


' Bom dia! Como posso ajudar?'

In [36]:
count_tokens(
    conversation_bufw, 
    "Meu interesse aqui é explorar o potencial de integração de Large Language Models com conhecimento externo."
)

Spent a total of 270 tokens


' Entendo. Estou familiarizado com Large Language Models e como eles podem ser usados para melhorar a compreensão de linguagem natural. Estou interessado em saber como esses modelos podem ser integrados com conhecimento externo. Existem várias abordagens para fazer isso, como a incorporação de informações externas em modelos de linguagem natural, a incorporação de conhecimento em modelos de linguagem natural e a incorporação de conhecimento em modelos de aprendizado profundo.'

In [37]:
count_tokens(
    conversation_bufw, 
    "Eu só quero analisar as diferentes possibilidades. O que você pode pensar?"
)

Spent a total of 538 tokens


' Existem várias maneiras de integrar conhecimento externo com Large Language Models. Uma abordagem é usar modelos de linguagem natural para incorporar informações externas. Isso pode ser feito adicionando informações externas ao modelo de linguagem natural, como dados de texto, imagens, vídeos ou outros dados. Outra abordagem é usar modelos de aprendizado profundo para incorporar conhecimento externo. Isso pode ser feito adicionando informações externas ao modelo de aprendizado profundo, como dados de texto, imagens, vídeos ou outros dados. Além disso, também é possível usar modelos de linguagem natural para incorporar conhecimento externo. Isso pode ser feito adicionando informações externas ao modelo de linguagem natural, como'

In [38]:
count_tokens(
    conversation_bufw, 
    "Quais tipos de fonte de dados podem ser usados para fornecer contexto ao modelo?"
)


Spent a total of 634 tokens


' Os dados externos usados para fornecer contexto ao modelo podem ser de vários tipos, como dados de texto, imagens, vídeos, áudio, dados de sensores, dados de localização, dados de redes sociais, dados de jogos, dados de saúde, dados de finanças, dados de clima, dados de tráfego, dados de mercado, dados de notícias, dados de blogs, dados de fóruns, dados de comentários, dados de análise de sentimentos, dados de análise de tendências, dados de análise de comportamento, dados de análise de conteúdo, dados de análise de linguagem, dados de análise de imagem, dados de análise de vídeo, dados de análise de áudio, dados de análise de dados e outros.'

In [39]:
count_tokens(
    conversation_bufw, 
    "Qual é o meu objetivo novamente?"
)

Spent a total of 387 tokens


' O seu objetivo é usar dados externos para fornecer contexto ao modelo.'

<font color="orange">Como podemos ver, ele efetivamente `'esqueceu'` o que conversamos na primeira interação. Vamos ver o que ele `'lembra'`. <font color="yellow">Dado que definimos k como 1, esperamos que ele se lembre apenas da última interação.</font>

Precisamos acessar um método especial aqui, pois neste tipo de memória, o buffer é passado primeiro por este método para ser enviado posteriormente para o LLM.</font>

In [40]:
bufw_history = conversation_bufw.memory.load_memory_variables(
    inputs=[]
)['history']


In [41]:
print(bufw_history)

Human: Qual é o meu objetivo novamente?
AI:  O seu objetivo é usar dados externos para fornecer contexto ao modelo.


<font color="pink">Faz sentido.

No lado positivo, estamos encurtando a duração da conversa em comparação com o `buffer de memória sem janela`:</font>

In [42]:
print(
    f'Comprimento da conversação da memória do buffer (ConversationBufferMemory): {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Comprimento da conversa de memória resumida (ConversationSummaryMemory): {len(tokenizer.encode(conversation_sum.memory.buffer))}\n'
    f'Comprimento da conversação da memória da janela de buffer (ConversationBufferWindowMemory): {len(tokenizer.encode(bufw_history))}'
)


Comprimento da conversação da memória do buffer (ConversationBufferMemory): 614
Comprimento da conversa de memória resumida (ConversationSummaryMemory): 256
Comprimento da conversação da memória da janela de buffer (ConversationBufferWindowMemory): 44


<font color="yellow">Nota Prática:</font> 


Estamos usando `k=1` aqui para fins ilustrativos, na maioria das aplicações do mundo real você precisaria de um valor mais alto para `k`.

## Mais tipos de memória!

Dado que já entendemos a memória, apresentaremos mais alguns tipos de memória aqui e esperamos que uma breve descrição seja suficiente para entender sua funcionalidade subjacente.


### <font color="pink">ConversationSummaryBufferMemory</font>

`Recurso principal:` a memória de resumo da conversa mantém um resumo das primeiras partes da conversa enquanto retém uma lembrança bruta das últimas interações.


### <font color="pink">ConversationKnowledgeGraphMemory</font>

Este é um tipo de memória super legal que foi introduzido recentemente. Baseia-se no `conceito de grafo de conhecimento` que reconhece diferentes entidades e as conecta em pares com um predicado resultando em trigêmeos `(sujeito, predicado, objeto)`. Isso nos permite compactar muitas informações em trechos altamente significativos que podem ser inseridos no modelo como contexto. Se você quiser entender esse tipo de memória com mais profundidade, confira esta postagem no [blog](https://apex974.com/articles/explore-langchain-support-for-knowledge-graph).


`Característica chave:` <font color="red">a memória do `grafo de conhecimento` da conversação mantém um `grafo de conhecimento` de todas as ENTIDADES que foram mencionadas nas interações junto com suas `relações SEMÂNTICAS`.</font>

In [43]:
# Você tem que instalar esta Biblioteca:

#%pip install -qU networkx

Note: you may need to restart the kernel to use updated packages.


In [44]:
conversation_kg = ConversationChain(
    llm=llm, 
    memory=ConversationKGMemory(llm=llm)
)


In [45]:
count_tokens(
    conversation_kg, 
    "Meu nome é Eddy e gosto muito de melancia!"
)

Spent a total of 1242 tokens


' Olá Eddy! É um prazer conhecê-lo. Eu sou um AI e estou aqui para ajudar. Você gosta de melancia? É uma fruta muito saborosa!'

<font color="orange">A memória mantém um `grafo de conhecimento` de tudo o que aprendeu até agora.</font>

In [46]:
conversation_kg.memory.kg.get_triples()

[('Eddy', 'Eddy', 'is named'), ('Eddy', 'melancia', 'likes')]

### <font color="pink">ConversationEntityMemory</font>


`Característica chave:` a memória das entidades de conversação guarda uma recordação das principais ENTIDADES MENCIONADAS, juntamente com os seus *atributos específicos*.

A maneira como isso funciona é bastante semelhante ao `ConversationKnowledgeGraphMemory`, você pode consultar os documentos se quiser vê-lo em ação.

# <font color="green">O que mais podemos fazer com a memória?</font>


Existem várias coisas legais que podemos fazer com a memória no `LangChain`. Pudermos:

🥸 implementar nosso próprio módulo de memória customizada

🥸 usar vários módulos de memória na mesma cadeia

🥸 combinar agentes com memória e outras ferramentas

Para mais detalhes dá uma olhada [AQUI](https://python.langchain.com/en/latest/modules/memory/how_to_guides.html)!